In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import warnings
from tensorflow.keras.layers import Input, BatchNormalization, LSTM, Dense, Reshape ,Dropout, Flatten
from tensorflow.keras.applications import ResNet50
from keras import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import roc_auc_score


<span style='background-color:#fff5b1'>data_dir는 사용자에 맞게 변경 필요</span>
------------------------------------------------

In [ ]:
data_dir = 'C:/Users/kungm/Desktop/ETRI/example/'

Model: CNN + RNN
============
 - CNN(ResNet50)
 - RNN(LSTM)

In [2]:
def build_model():
    #cnn
    inp = Input(shape=(224,224,3)) # (channels,height,width)
    #include_top=False는 가장 상단의 fully connected계층들을 포함 시키지 않음
    resnet = ResNet50(input_tensor=inp, include_top=False, weights='imagenet', pooling='avg')
    resnet_output = resnet.output

    #Fitting 문제 방지
    resnet_batch_norm = BatchNormalization(axis=1)(resnet_output)
    
    #lstm 입층력 층에 맞추기 위해 reshape
    lstm_input = Reshape((1, 2048))(resnet_batch_norm)
    
    #lstm
    lstm0 = LSTM(64,activation='relu',return_sequences=True)(lstm_input)
    lstm_batch_norm0 = BatchNormalization(axis=1)(lstm0)

    lstm1 = LSTM(32,activation='relu',return_sequences=True)(lstm_batch_norm0)
    lstm_batch_norm1 = BatchNormalization(axis=1)(lstm1)

    lstm2 = LSTM(16,activation='relu',return_sequences=True)(lstm_batch_norm1)


    #no matching domension issue 해결 위함
    lstm_f = Flatten()(lstm2)
    dense = Dense(16, activation='relu')(lstm_f)
    batch = BatchNormalization(axis=1)(dense)
    dropout = Dropout(0.3)(batch)
    output= Dense(7, activation='softmax')(dropout)
    
    CNN_RNN = Model(inputs = inp, outputs = output)
    

    return CNN_RNN

Load train dataset
==========

In [3]:
data_train={}
data_train['ID']=[]
data_train['path']=[]
data_train['emotion']=[]


with open(data_dir + 'train_final.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    data_train = pickle.load(f)
    

ds_train = pd.DataFrame.from_dict(data_train)



Model training
==========

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
 try:
     tf.config.experimental.set_memory_growth(gpus[0], True)
 except RuntimeError as e:
     print(e)

warnings.filterwarnings('ignore')
keras.backend.clear_session()

adam = tf.keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


CNN_RNN = build_model()

CNN_RNN.compile(
    optimizer = adam,
    loss = 'categorical_crossentropy',
    metrics=['accuracy'])

######################################## 5-fold 교차 검증(cross-validation) ######################################## 

SPLITS = 5
skf = StratifiedKFold(n_splits = SPLITS)
n_iter = 0

features = ds_train.iloc[:,1:2]
label = pd.DataFrame(ds_train['emotion'])

score_list = []
for train_idx, val_idx in skf.split(features, label):
    n_iter += 1
    print(f'--------------------{n_iter}번째 KFold-------------------')
    print(f'train_idx_len : {len(train_idx)} / val_idx_len : {len(val_idx)}')

    label_train = label.iloc[train_idx]
    label_val = label.iloc[val_idx]

    X_train, X_val = features.iloc[train_idx, :], features.iloc[val_idx, :]
    y_train, y_val = label.iloc[train_idx,:], label.iloc[val_idx,:]
    
    img_train=[]
    label_train=[]
    img_val=[]
    label_val=[]

    for index in train_idx:
        img = X_train['path'][index]
        img_train.append(img)
        emotion = y_train['emotion'][index]
        label_train.append(emotion)
        
    for index in val_idx:
        img = X_val['path'][index]
        img_val.append(img)
        emotion = y_val['emotion'][index]
        label_val.append(emotion)
        
        
    label_train= tf.keras.utils.to_categorical(label_train)
    label_val= tf.keras.utils.to_categorical(label_val)  
    
    img_train=np.array(img_train)
    label_train=np.array(label_train)
    img_val=np.array(img_val)
    label_val=np.array(label_val)


    with tf.device('/device:GPU:0'):
        history = CNN_RNN.fit(
            img_train,
            label_train,
            batch_size=16,
            epochs=60, #epochs 40,50,100
            verbose=1)

    preds = CNN_RNN.predict(img_val)
    preds_labels = np.argmax(preds, axis=1)
    y_val_labels = np.argmax(label_val, axis=1)
    score = accuracy_score(y_val_labels, preds_labels)

    print(f'{n_iter}번째 단일 accuracy_score:{score}')
    score_list.append(score)


print('======================================================')
print(f'최종 평균 accuracy_socre : {sum(score_list)/len(score_list)}')


Load test dataset
==========

In [5]:
data_test={}
data_test['ID']=[]
data_test['path']=[]
data_test['emotion']=[]



with open(data_dir + 'test_final.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    data_test = pickle.load(f)

ds_test = pd.DataFrame.from_dict(data_test)    

Predict test data
==========

In [6]:
from keras.models import load_model


# 위의 모델 학습을 돌리지 않고, 완성된 모델을 가져와 predict test data
CNN_RNN = load_model(data_dir + 'CNN_RNN_fold_epoch60_e4.h5')

In [ ]:
feature_test=[]
label_test=[]


        
for i in range(len(data_test['path'])):
    feature = data_test['path'][i]
    feature_test.append(feature)
    emotion = data_test['emotion'][i]
    label_test.append(emotion)
    
label_test= tf.keras.utils.to_categorical(label_test)

feature_test=np.array(feature_test)
label_test=np.array(label_test)


y_test  = label_test
y_preds_test = CNN_RNN.predict(feature_test)
print('test', roc_auc_score(y_test[:len(y_preds_test)], y_preds_test))